In [1]:
import json
from collections import defaultdict
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
from collections import deque
import itertools
d = deque()
d.append(1)
d.append(2)
d.append(3)
#d.popleft(), d

# Сравнение разных файлов

In [3]:
df = pd.read_csv("../../Data/Raw/merged_with_labels_11_patterns.csv", sep=";")
print(len(df[df['x'].isna()]), "nan texts")
df = df[~df['x'].isna()]

df_2 = pd.read_csv("../../Data/Raw/generated_with_vector.csv", sep=";")
print(len(df_2[df_2['x'].isna()]), "nan texts")
df_2 = df_2[~df_2['x'].isna()]

df_3 = pd.read_csv("../../Data/Raw/12052022_one_cmd.csv", sep=";")
df_3.drop("delayed", axis=1, inplace=True)

df_4 = pd.read_csv("../../Data/Raw/one_cmd_v3_03062022.csv", sep=";")
df_4.drop("delayed", axis=1, inplace=True)

df_5 = pd.read_csv("../../Data/Raw/one_cmd_v4_05072022.csv", sep=";")
df_5.drop("delayed", axis=1, inplace=True)

0 nan texts
10 nan texts


In [4]:
df_3

,x,y_name,y,action,direction,meters,degshours,object1,nearest,relation1,object2,relation2,object3,self,gaze,type
0,обойди окрестности по кругу радиусом 58m,circle,9,0,0,10,0,0,0,0,0,0,0,0,0,generator
1,начинай патрулирование по кругу радиуса 27 метров,circle,9,0,0,8,0,0,0,0,0,0,0,0,0,generator
2,разведка по орбите размером 7м,circle,9,0,0,4,0,0,0,0,0,0,0,0,0,generator
3,неси дозор по кругу 17m,circle,9,0,0,6,0,0,0,0,0,0,0,0,0,generator
4,неси патрульную службу по орбите радиуса 18m,circle,9,0,0,7,0,0,0,0,0,0,0,0,0,generator
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,исследуй территорию,simple,0,0,0,0,0,0,0,0,0,0,0,0,0,students
20048,Проведай обстановку,simple,0,0,0,0,0,0,0,0,0,0,0,0,0,students
20049,Давай ка обойди участок,simple,0,0,0,0,0,0,0,0,0,0,0,0,0,students
20050,Быстро в разведку,simple,0,0,0,0,0,0,0,0,0,0,0,0,0,students


In [11]:
df["type"].unique()

array(['new_generator', 'students', 'crowdsource', 'generator'],
      dtype=object)

In [5]:
df[~df["type"].isin(["generator", 'new_generator'])]

,x,y_name,y,action,direction,meters,degshours,object1,nearest,relation1,object2,relation2,object3,self,gaze,type
1,постереги,simple,0,0,0,0,0,0,0,0,0,0,0,0,0,students
2,дай по тормозам,simple,0,1,0,0,0,0,0,0,0,0,0,0,0,students
3,ваша остановка,simple,0,1,0,0,0,0,0,0,0,0,0,0,0,students
4,припаркуйся,simple,0,1,0,0,0,0,0,0,0,0,0,0,0,students
5,приехали,simple,0,1,0,0,0,0,0,0,0,0,0,0,0,students
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49457,"возле лесо стоит человек, за лесом есть повале...",relation2,8,6,0,0,0,3,0,2,4,1,6,0,0,crowdsource
49458,"Отправляйся к спиленному деревуза лесополосой,...",relation2,8,6,0,0,0,3,0,2,4,1,6,0,0,crowdsource
49459,"Шагай в сторону дерева, лежащего позади леса, ...",relation2,8,6,0,0,0,3,0,2,4,1,6,0,0,crowdsource
49460,"Рядом с человеком есть лес, за которым повален...",relation2,8,6,0,0,0,3,0,2,4,1,6,0,0,crowdsource


In [13]:
# принтануть команды из 1 слова
#for ir, row in df_5[df_5["x"].map(str.split).map(len)==1].iterrows():
#    print(row.values)

['разворачивайся ' 'degree_around' 3 5 4 0 12 0 0 0 0 0 0 0 0 'generator']
['разверни ' 'degree_around' 3 5 4 0 12 0 0 0 0 0 0 0 0 'generator']
['развернись ' 'degree_around' 3 5 4 0 12 0 0 0 0 0 0 0 0 'generator']
['разворот ' 'degree_around' 3 5 4 0 12 0 0 0 0 0 0 0 0 'generator']
['повернись' 'direction' 1 3 3 0 0 0 0 0 0 0 0 0 0 'generator']
['собирайся' 'direction' 1 2 1 0 0 0 0 0 0 0 0 0 0 'generator']
['причаль' 'direction' 1 2 1 0 0 0 0 0 0 0 0 0 0 'generator']
['выскочи' 'direction' 1 2 1 0 0 0 0 0 0 0 0 0 0 'generator']
['заворчивай' 'direction' 1 3 3 0 0 0 0 0 0 0 0 0 0 'generator']
['подымайся' 'direction' 1 2 1 0 0 0 0 0 0 0 0 0 0 'generator']
['поворачивай' 'direction' 1 3 3 0 0 0 0 0 0 0 0 0 0 'generator']
['проходи' 'direction' 1 2 1 0 0 0 0 0 0 0 0 0 0 'generator']
['откатись' 'direction' 1 2 1 0 0 0 0 0 0 0 0 0 0 'generator']
['выдвигайся' 'direction' 1 2 1 0 0 0 0 0 0 0 0 0 0 'generator']
['броссься' 'direction' 1 2 1 0 0 0 0 0 0 0 0 0 0 'generator']
['давай' 'direct

In [48]:
from sklearn.utils.class_weight import compute_class_weight

array([  0.55205225,   0.47469796,   1.66978482,  13.04178338,
         0.45834222,   0.81979986,   0.83228065,   0.80916174,
         0.66646388,   0.93003629,   1.02066131, 173.51242236,
       285.05612245, 362.7987013 ])

In [56]:
for col in ['action', 'direction', 'meters', 'degshours',
       'object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3',
       'self', 'gaze']:
    print(col)
    value_counts = df_5[col].value_counts()
    value_counts.sort_index(inplace=True)
    #print(list(np.round(10*np.clip(1 - value_counts / max(value_counts)+0.1, 0, 1).values, 2)))
    print(list(np.round(compute_class_weight("balanced", classes=df_5[col].unique(), y=df_5[col]), 2)))

action
[0.55, 0.47, 1.67, 13.04, 0.46, 0.82, 0.83, 0.81, 0.67, 0.93, 1.02, 173.51, 285.06, 362.8]
direction
[0.16, 3.28, 1.59, 3.15, 3.24, 3.28, 3.3, 2.21, 7.43]
meters
[2.5, 4.41, 4.3, 4.41, 4.35, 4.42, 4.43, 4.35, 4.28, 4.38, 4.59, 2.54, 0.1]
degshours
[0.05, 6.46, 6.23, 6.61, 6.61, 5.96, 6.94, 6.01, 6.23, 5.99, 6.11, 6.39, 6.48, 5.79, 6.53, 6.57, 5.87, 6.48, 7.23, 6.59, 6.21, 6.46, 6.9, 6.9, 6.07]
object1
[0.57, 0.18, 1.45, 1.5, 1.53, 1.57, 1.46, 1.49, 1.56, 1.5, 1.56, 1.56, 1.52, 1.58, 1.54, 1.59, 2.21]
nearest
[0.55, 5.34]
relation1
[0.21, 3.94, 3.84, 3.88, 3.92, 3.82]
object2
[0.08, 4.29, 4.54, 4.44, 4.04, 4.54, 4.28, 4.47, 4.43, 4.23, 4.25, 4.19, 4.52, 4.23, 4.45]
relation2
[0.19, 7.93, 7.86, 7.44, 7.44, 8.18]
object3
[0.07, 9.55, 8.58, 8.78, 8.7, 8.66, 8.93, 8.15, 8.41, 8.45, 9.48, 8.28, 8.87, 8.74, 8.33]
self
[0.22, 7.66, 9.23, 9.06, 7.69]
gaze
[0.55, 5.33]


In [13]:
test_df = df[df["type"].isin(['students', 'crowdsource'])]
for y_label in test_df["y"].unique():
    print(y_label)
    for c in test_df.columns:
        if c in ["x", "y_name", "y", "type"]:
            continue
        uniq = test_df[test_df["y"]==y_label][c].unique()
        if len(uniq)==1 and uniq[0]==0:
            continue
        #active_labels_for_y[int(y_label)].append(c)
        print(c, sorted(uniq))
    print()

0
action [0, 1]

1
action [2, 3]
direction [1, 2, 3, 4]

5
action [6]
object1 [1, 2, 3, 4, 5, 6]

7
action [6]
object1 [1, 2, 3, 4, 5, 6]
relation1 [1, 2]
object2 [1, 2, 3, 4, 5, 6]

8
action [6]
object1 [1, 2, 3, 4, 5, 6]
relation1 [1, 2]
object2 [1, 2, 3, 4, 5, 6]
relation2 [1, 2]
object3 [1, 2, 3, 4, 5, 6]



In [7]:
len(df_4["action"].unique())

13

In [4]:
df_4.columns

Index(['x', 'y_name', 'y', 'action', 'direction', 'meters', 'degshours',
       'object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3',
       'self', 'gaze', 'type'],
      dtype='object')

In [9]:
df = df_4

In [4]:
sum([13,9, 13, 25, 15, 2, 6, 15, 6, 15, 5, 2])

126

In [7]:
for c in df_4.columns:
    if c in ['x', "y_name"]:
        continue
    print(c, len(df_4[c].unique()))

y 12
action 13
direction 9
meters 13
degshours 25
object1 15
nearest 2
relation1 6
object2 15
relation2 6
object3 15
self 5
gaze 2
type 1


In [3]:
# Количество классов лейблов на каждый лейбл
for c in ["y", "action", "direction", "meters", "degshours", "object1", "nearest", "relation1", "object2", "relation2", "object3", "gaze"]:
    if c not in df_2.columns:
        print(c, len(df[c].unique()), 0, len(df_3[c].unique()))
        continue
    print(c, len(df[c].unique()),len(df_2[c].unique()), len(df_3[c].unique()))

y 12 9 12
action 10 7 10
direction 9 9 9
meters 13 9 13
degshours 25 25 25
object1 15 15 15
nearest 2 2 2
relation1 6 6 6
object2 15 15 15
relation2 6 6 6
object3 15 15 15
gaze 2 0 2


In [28]:
# Сохранить все возможные комбинации лейблов
all_labels = ['y', 'action', 'direction', 'meters', 'degshours','object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3', 'self', 'gaze']
all_possible_combinations = df.loc[df["type"].isin(["new_generator", "generator"]), all_labels].drop_duplicates(subset=all_labels)
print(len(all_possible_combinations), "комбинаций в 1 датасете")
all_possible_combinations_3 = df_3.loc[:, all_labels].drop_duplicates(subset=all_labels)
print(len(all_possible_combinations_3), "комбинаций во 2 датасете")
combinations_1_set = set()
for ir, row in all_possible_combinations.iterrows():
    combinations_1_set.add(tuple(row.values))
combinations_3_set = set()
for ir, row in all_possible_combinations_3.iterrows():
    combinations_3_set.add(tuple(row.values))
print(len(combinations_1_set - combinations_3_set), "комбинаций есть в первом, но нет в 3")
print(len(combinations_3_set - combinations_1_set), "комбинаций есть в 3, но нет в 1")
#all_possible_combinations.to_csv("../Data/Interim/possible_combinations.csv", index=False)
all_possible_combinations = pd.concat([all_possible_combinations, all_possible_combinations_3])
all_possible_combinations.drop_duplicates(subset=all_labels).to_csv("../../Data/Interim/possible_combinations.csv", index=False)
#all_possible_combinations[all_possible_combinations!=0] = 1
#all_possible_combinations.drop_duplicates(subset=[c for c in all_labels if c!="y"]).to_csv("../../Data/Interim/possible_combinations_activations.csv", index=False)
all_possible_combinations.drop("y", axis=1, inplace=True)
all_attributes = [c for c in all_labels if c not in ["y", "action"]]
all_possible_combinations[all_possible_combinations[all_attributes]!=0]=1
all_possible_combinations.drop_duplicates(inplace=True)
all_possible_combinations.to_csv("../../Data/Interim/possible_combinations_of_attributes_for_actions.csv", index=False)

11587 комбинаций в 1 датасете
4360 комбинаций во 2 датасете
10405 комбинаций есть в первом, но нет в 3
3178 комбинаций есть в 3, но нет в 1


In [25]:
all_possible_combinations = pd.read_csv("../../Data/Interim/possible_combinations.csv")
all_possible_combinations_2 = pd.read_csv("../../Data/Interim/possible_combinations_backup.csv")
tuples_1 = set()
for row in all_possible_combinations.values:
    tuples_1.add(tuple(row.tolist()))
tuples_2 = set()
for row in all_possible_combinations_2.values:
    tuples_2.add(tuple(row.tolist()))
len(tuples_2 - tuples_1), len(tuples_1 - tuples_2)

(9814, 7602)

In [51]:
# для какого Y какие классы других лейблов возможны
active_labels_for_y = defaultdict(dict)
for y_label in df["y"].unique():
    print(y_label)
    for c in df.columns:
        if c in ["x", "y_name", "y", "type"]:
            continue
        uniq = df[df["y"]==y_label][c].unique()
        if len(uniq)==1 and uniq[0]==0:
            continue
        #active_labels_for_y[int(y_label)].append(c)
        active_labels_for_y[int(y_label)][c] = [int(x) for x in sorted(uniq)]
        print(c, sorted(uniq))
    print()

0
action [0, 1]

1
action [2, 3]
direction [1, 2, 3, 4, 5, 6, 7, 8]

2
action [4]
direction [1, 2, 3, 4, 5, 6, 7, 8]
meters [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

3
action [5]
direction [3, 4, 5, 6, 7, 8]
degshours [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]

4
action [5]
direction [3, 4, 5, 6, 7, 8]
degshours [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]

5
action [6, 7, 8, 9]
object1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

6
action [6]
object1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
nearest [1]

7
action [6]
object1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
relation1 [1, 2, 3, 4, 5]
object2 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

8
action [6]
object1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
relation1 [1, 2, 3, 4, 5]
object2 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
relation2 [1, 2, 3, 4, 5]
object3 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

9
mete

In [55]:
# для какого Y какие классы других лейблов возможны
active_labels_for_y = defaultdict(dict)
for y_label in sorted(df_3["y"].unique()):
    print(y_label)
    for c in df_3.columns:
        if c in ["x", "y_name", "y", "type"]:
            continue
        uniq = df_3[df_3["y"]==y_label][c].unique()
        if len(uniq)==1 and uniq[0]==0:
            continue
        #active_labels_for_y[int(y_label)].append(c)
        active_labels_for_y[int(y_label)][c] = [int(x) for x in sorted(uniq)]
        print(c, sorted(uniq))
    print()

0
action [0, 1]

1
action [2, 3]
direction [1, 2, 3, 4, 5, 6, 7, 8]

2
action [4]
direction [1, 2, 3, 4, 5, 6, 7, 8]
meters [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

3
action [5]
direction [3, 4, 5, 6, 7, 8]
degshours [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]

4
action [5]
direction [3, 4, 5, 6, 7, 8]
degshours [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]

5
action [6, 7, 8, 9]
object1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

6
action [6, 7, 8, 9]
object1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
nearest [1]

7
action [6, 7, 8, 9]
object1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
relation1 [1, 2, 3, 4, 5]
object2 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

8
action [6, 7, 8, 9]
object1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
relation1 [1, 2, 3, 4, 5]
object2 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
relation2 [1, 2, 3, 4, 5]
object3 [1, 2, 3, 4, 5, 6, 7, 8, 9, 

In [57]:
with open("../../Data/Interim/active_labels_for_y.json", "w") as f:
    json.dump(active_labels_for_y, f)

In [62]:
print(len(set(df_2["x"])),  len(set(df["x"])), len(set(df_3["x"])))
print(len(set(df["x"]) - set(df_2["x"])), len(set(df_2["x"]) - set(df["x"])))
print(len(set(df["x"]) - set(df_3["x"])), len(set(df_3["x"]) - set(df["x"])))

41175 67693 20044
60232 33714
66497 18848


In [105]:
# количество примеров классов для каждого лейбла
for c in df_3.columns:
    if c in ["x", "y_name", "type"]:
        continue
    print(df_3[c].value_counts())

7     2000
8     2000
2     1999
3     1987
9     1979
11    1915
6     1850
4     1823
10    1810
5     1484
1     1116
0       89
Name: y, dtype: int64
6    3827
5    3810
8    2485
9    2481
7    2266
0    2027
4    1999
2     894
3     222
1      41
Name: action, dtype: int64
0    13127
3     1771
1     1006
8      796
4      795
5      777
6      768
7      760
2      252
Name: direction, dtype: int64
0     16074
8       355
11      352
5       341
12      340
4       339
10      330
3       327
9       327
7       326
2       325
1       321
6       295
Name: meters, dtype: int64
0     16242
11      188
8       175
24      173
21      170
13      168
23      168
2       167
14      166
17      164
6       161
7       161
1       161
10      160
18      159
5       158
19      156
22      155
3       152
9       152
15      149
20      138
16      137
4       137
12      135
Name: degshours, dtype: int64
0     9595
13     774
3      774
10     773
6      762
2      758
5      751


# Вытащить уникальные примеры для разных случаев

In [24]:
from collections import defaultdict
examples = defaultdict(list)
examples_uniqflags = defaultdict(list)
for ir, row in df[df["type"].isin(["students", "crowdsource"])].iterrows():
    labels_tuple = tuple(row.loc[[c for c in df.columns if c not in ["x", "y_name", "type"]]].values.tolist())
    labels_flags = tuple([1 if v!=0 else v for v in labels_tuple])
    if len(examples[labels_tuple])<1:
        examples[labels_tuple].append(row["x"])
    if len(examples_uniqflags[labels_flags])<1:
        examples_uniqflags[labels_flags].append(row["x"])
for ir, row in df.iterrows():
    labels_tuple = tuple(row.loc[[c for c in df.columns if c not in ["x", "y_name", "type"]]].values.tolist())
    labels_flags = tuple([1 if v!=0 else v for v in labels_tuple])
    if len(examples[labels_tuple])<1:
        examples[labels_tuple].append(row["x"])
    if len(examples_uniqflags[labels_flags])<1:
        examples_uniqflags[labels_flags].append(row["x"])

In [26]:
with open("../Data/Interim/service_test_examples.txt", "w") as f:
    for k, v in examples.items():
        for phrase in v:
            f.write(phrase+"\n")
with open("../Data/Interim/service_test_examples_short.txt", "w") as f:
    for k, v in examples_uniqflags.items():
        for phrase in v:
            f.write(phrase+"\n")